In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
df_test = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
df_train.head()

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
import cv2
from PIL import Image
from tensorflow import keras
from keras.layers import Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from tensorflow.keras.regularizers import l2
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras import optimizers


In [ ]:
# Flatten the 'tags' column of the training data-set into a list
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list (set(flatten([l.split (' ') for l in df_train['tags'].values])))

In [ ]:
label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}
label_map

In [ ]:
# Loading the training images

x_train, y_train = [], []

for img_name, label in tqdm(df_train.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(img_name))
    targets = np.zeros(17)
    for tag in label.split(' '):
        targets[label_map[tag]] = 1 
    x_train.append(cv2.resize(arr, (64, 64)))
    y_train.append(targets)

In [ ]:
# Loading the testing images
x_test = []
for img_name, label in tqdm(df_test.values[0:40669], miniters=1000):
    img = cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(img_name))
    x_test.append(cv2.resize(img, (64, 64)))
    
for img_name, label in tqdm(df_test.values[40669:], miniters=1000):
    img = cv2.imread('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(img_name))
    x_test.append(cv2.resize(img, (64, 64)))

In [ ]:
import gc
gc.collect()

In [ ]:
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16)/255.
x_test  = np.array(x_test, np.float16)/255.

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, shuffle = True, random_state = 1)


print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

In [ ]:
gc.collect()

In [ ]:
# function for fbeta evaluation metric

from keras import backend as K
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import regularizers

model = Sequential()
model.add(BatchNormalization(input_shape=(64, 64,3)))
model.add(Conv2D(32, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
        
model.add(Conv2D(128, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
        
model.add(Conv2D(256, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
        
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))




In [ ]:
# defining a vgg16
from tensorflow.keras.applications.resnet import ResNet152
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# add a global spatial average pooling layer
x = base_model.output
x=Flatten()(x)
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)
predictions = Dense(17, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)



learn_rate=0.0001
opt  = optimizers.Adam(lr=learn_rate)
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics = [fbeta])
model.summary()


In [ ]:
# setting an early callback
epochs = 20
 
history = History()
callbacks = [history, 
             EarlyStopping(monitor='val_loss', patience=3, verbose=1),
             ModelCheckpoint(filepath='weights/weights.best.hdf5', verbose=1, save_best_only=True, 
                             save_weights_only=True, mode='auto')]


In [ ]:
# performing image augmentation
datagen = ImageDataGenerator ( horizontal_flip =True ,
vertical_flip =True ,
zoom_range =0.2,
rotation_range =90 ,
fill_mode ='reflect')

In [ ]:
gc.collect()

In [ ]:
# Fit the model on the batches generated by datagen.flow().
model .fit_generator(datagen.flow(x_train,
y_train,batch_size =24),steps_per_epoch =len(x_train)/32 ,
validation_data = datagen.flow ( x_val,y_val,batch_size =24),
validation_steps =len(x_val)/32 ,epochs =epochs ,callbacks = callbacks ,verbose =1)
   

In [ ]:
gc.collect()

In [ ]:
yfull_test = []
p_test = model.predict(x_test, batch_size =128, verbose=2)
yfull_test.append(p_test)

In [ ]:
gc.collect()

In [ ]:
result = np.array(yfull_test [0])
for i in range(1, len(yfull_test)):
    result += np.array(yfull_test[i])
result = pd.DataFrame(result, columns = labels)
result

In [ ]:
preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))
    


In [ ]:
df_test['tags'] = preds
df_test.to_csv('Final_output(vgg).csv', index=False)

Achieved a F-Beta score of 0.90833 using VGG16